In [1]:
import re
import random
import pandas as pd

from tqdm import tqdm
from koeda import EasyDataAugmentation, AEasierDataAugmentation
from googletrans import Translator

In [2]:
import time

def timeit(func):
    """이 데코레이터는 함수의 실행 시간을 측정합니다."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 시작 시간 측정
        result = func(*args, **kwargs)
        end_time = time.time()    # 종료 시간 측정
        elapsed_time = end_time - start_time
        print(f"'{func.__name__}' 함수 실행 시간: {elapsed_time:.4f}초")
        return result
    return wrapper

In [3]:
df = pd.read_csv('../dataset/cleaned_train.csv')

In [4]:
idx = random.randint(0, 12000)
sample = df.iloc[idx]['dialogue']

In [5]:
@timeit
def augment_text_data_with_EDA(text: str, repetition=1):
    eda = EasyDataAugmentation(morpheme_analyzer="Mecab")
    
    pattern = r'#.*?#:?'
    
    lines = text.split('\n')
    augmented_texts = []
    
    for line in lines:
        matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, line)]
        cleaned_line = line
        for start, end, match in matches:
            cleaned_line = cleaned_line.replace(match, "")
        
        result = eda(cleaned_line, p=(0.1, 0.1, 0.1, 0.1), repetition=repetition)
        
        for start, end, match in matches:
            result = result[:start] + match + result[start:]
        
        augmented_texts.append(result.strip())

    return augmented_texts

In [6]:
# print(sample)
augment_text_data_with_EDA(sample)

'augment_text_data_with_EDA' 함수 실행 시간: 0.0072초


['#Person1#:오, 정말 신나네요. 그럼, 어디에서 결혼하고 싶으세요? 흙덩이 알아요, 마련 알아요, 알아요 동정 . 음, 산 위에서요. 아시 임 죠, 그 아름다운 폭포? 폭포 옆에서 결혼하는 건 어때요?',
 '#Person2#:음, 잘 모르겠네요.',
 '#Person1#:오 그런 말 하지 마세요. 정말 좋을 거예요. 너무 아름답, 평화롭고, 정말 로맨틱하죠. 부탁이에요.',
 '#Person2#:잘 모르겠어요. 제가 야생화 32핵에 알레르기가 있잖아요.',
 '#Person1#:괜찮을 거예요. 알레르기 을 먹으면 돼요.',
 '#Person2#:. 그리고 탄탄 심한 발진이 나타날 수도 있어요.',
 '#Person1#:괜찮을 거예요. 가자고요. 음, 우리가 좋은 걸 생각해낼 호수 있을 거예요. 가자고요. 부탁이에요.',
 '#Person2#:우우우우리 부모님 집 명가 뒷마당에서 결혼하면 장 어떨까요? [ 뭐라고요? ] 그래요. 개들을 몇 시간만 옮기 체모 면 돼요.',
 '#Person1#:아니요! 그런 생각도 하지 마세요. 그들의 뒷마당은 정말 쓰.쓰(그녀가 말하려는 것은, 쓰레기 같다는 것). 상태가. 상태가 안 좋아요. 그리고 그들의 집은 고속도로 바로 옆수확에요. 고속도로에서 반 블록밖에 안 떨어져 있어요. 너무 시끄러워요. 부탁수확에요. 몇 마일 떨어진 곳에 정말 좋은 리셉션 홀수확 있어요. 우리가 그곳에서 할 수 있어요. 알레르기 걱정은 안 해도 돼요.',
 '#Person2#:아, 그게 비싸 보이는데요!',
 '#Person1#:그렇게 비싼 건 아니에요. 조금 비싸긴 해요. 그리고, 그리고, 요정 봐요. 우리가 케이터링 서비스를 불러서, 그들 백금 이 새우와 랍스터 천수 표제 포 꼬리를 제공하고, 제 친구가 결혼했을 주간 때, 그들은 최고의 디저트를 가지고 있었어요. 고난 부탁이에요, 그리고, 아시다시피, 우리의 결혼식 아침 식사를 위해, 우리는 그 일본 레스토랑에 갈 수 부지 있어요 사상 . 기억이 안 나네요. 그게 미소 뭐였지요?'

In [7]:
@timeit
def augment_text_data_with_AEDA(text, repetition):
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(morpheme_analyzer="Okt", punctuations=[".", ",", "!", "?", ";", ":"])

    pattern = r'#.*?#:?'
    lines = text.split('\n')
    augmented_texts = []
    
    for line in lines:
        matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, line)]
        
        cleaned_line = line
        for start, end, match in matches:
            cleaned_line = cleaned_line.replace(match, "")
    
        result = aeda(text, p=0.1, repetition=repetition)
    
        for start, end, match in matches:
            result = result[:start] + match + result[start:]
        
        augmented_texts.append(result.strip())

    return augmented_texts

In [8]:
augment_text_data_with_AEDA(sample, 1)

KeyboardInterrupt: 

: 

In [ ]:
@timeit
def augment_text_data_with_BT(text, repetition):
    """입력된 문장에 대해서 BT를 통해 데이터 증강"""
    translator = Translator()
    result = []

    # 패턴 정의: #문자열#: 또는 #문자열# 추출
    pattern = r'#.*?#:?'
    matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, text)]
    
    cleaned_text = text
    for start, end, match in matches:
        cleaned_text = cleaned_text.replace(match, "")

    # 번역 실행 (한국어 > 영어 > 한국어)
    for i in range(repetition):
        translated = translator.translate(cleaned_text, src='ko', dest='en')
        re_translated = translator.translate(translated.text, src='en', dest='ko')
        augmented_text = re_translated.text
        
        for start, end, match in matches:
            augmented_text = augmented_text[:start] + match + augmented_text[start:]
        
        result.append(augmented_text)
    
    return result

In [ ]:
augment_text_data_with_BT(sample, 1)

In [ ]:
def augment_dataframe(df, repetition=1):
    augmented_rows = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Augmenting Data"):
        original_dialogue = row['dialogue']
        summary = row['summary']
        topic = row['topic']

        eda_augmented_texts = augment_text_data_with_EDA(original_dialogue, repetition)
        aeda_augmented_texts = augment_text_data_with_AEDA(original_dialogue, repetition)
        bt_augmented_texts = augment_text_data_with_BT(original_dialogue, repetition)

        for text in eda_augmented_texts + bt_augmented_texts + aeda_augmented_texts:
        # for text in eda_augmented_texts + bt_augmented_texts:
            augmented_rows.append({
                'dialogue': text,
                'summary': summary,
                'topic': topic
            })


    augmented_df = pd.DataFrame(augmented_rows)
    result_df = pd.concat([df, augmented_df], ignore_index=True)

    return result_df

In [ ]:
# augmented_df = augment_dataframe(df, repetition=1)

In [ ]:
# augmented_df.to_csv('../dataset/augmented_train.csv', index=False)